# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [33]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [39]:
# Filter and select relevant columns
df_salary = df.filter((df["SALARY_FROM"].isNotNull()) & (df["SALARY_FROM"] > 0))
pdf = df_salary.select("EMPLOYMENT_TYPE_NAME", "SALARY_FROM").toPandas()
pdf["EMPLOYMENT_TYPE_NAME"] = pdf["EMPLOYMENT_TYPE_NAME"].replace({
    "Part-time (â‰¤ 32 hours)": "Part-time (≤ 32 hours)"
})


# Create box plot
fig = px.box(
    pdf,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Employment Type",
    color_discrete_sequence=["#636EFA"]
)

# Customize fonts and layout
fig.update_layout(
    font_family="Arial",
    title_font_size=16,
    xaxis_title="Employment Type",
    yaxis_title="Salary From"
)

fig.show()


### Salary Distribution by Employment Type (Box Plot)

This box plot illustrates the distribution of salaries across different employment types.  
Full-time jobs (≥ 32 hours) show a wider salary range and higher maximum salaries, including significant outliers.  
In contrast, part-time positions exhibit lower median salaries and a more concentrated distribution.  

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 2nd question here
# 3: Salary Distribution by Industry (NAICS2)

# Filter out records with missing or zero salary
df_industry = df.filter((df["SALARY_FROM"].isNotNull()) & (df["SALARY_FROM"] > 0))

# Select necessary columns and convert to pandas
pdf_industry = df_industry.select("NAICS2_NAME", "SALARY_FROM").toPandas()

# Optional: handle null industry labels
pdf_industry["NAICS2_NAME"] = pdf_industry["NAICS2_NAME"].fillna("Unknown")

# Draw the box plot
fig = px.box(
    pdf_industry,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Industry (NAICS2)",
    color_discrete_sequence=["#00BFC4"]
)

# Customize styles
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Industry (NAICS2)",
    yaxis_title="Salary From",
    xaxis_tickangle=45  
)

fig.show()


### Salary Distribution by Industry (NAICS2)

This box plot shows how salaries vary across different industries, based on NAICS-2 classification codes.  
Industries such as "Administrative and Support" and "Information" display a wider salary range and include significant outliers, suggesting diverse roles with varying pay levels.  
On the other hand, sectors like "Retail Trade" and "Accommodation and Food Services" have more compressed salary distributions, indicating more consistent pay across positions.  


# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
from pyspark.sql.functions import to_date, col

# Convert string to proper date format
df_time = df.withColumn("POSTED_DATE", to_date(col("POSTED"), "M/d/yyyy"))

# Filter out null dates just in case
df_time = df_time.filter(df_time["POSTED_DATE"].isNotNull())

# Count postings per date
df_posting_counts = df_time.groupBy("POSTED_DATE").count().orderBy("POSTED_DATE")

# Convert to pandas
pdf_posting_counts = df_posting_counts.toPandas()

# Draw line chart
fig = px.line(
    pdf_posting_counts,
    x="POSTED_DATE",
    y="count",
    title="Job Posting Trends Over Time",
    markers=True,
    line_shape="linear"
)

fig.update_layout(
    font_family="Arial",
    title_font_size=20,
    xaxis_title="Date Posted",
    yaxis_title="Number of Job Postings",
    template="plotly_white"
)

fig.show()



### Job Posting Trends Over Time

This time series line chart illustrates fluctuations in the number of job postings over time.  
There are noticeable spikes and dips in activity, suggesting periodic surges in hiring—possibly aligned with business cycles or seasonal demands.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
from pyspark.sql.functions import col
import plotly.express as px

# Step 5: Count job titles and select top 10
df_title_count = df.groupBy("TITLE_NAME").count().orderBy(col("count").desc()).limit(10)

# Convert to Pandas for plotting
pdf_title_count = df_title_count.toPandas()

# Sort again in Pandas (optional for better display)
pdf_title_count = pdf_title_count.sort_values("count", ascending=True)  # 为了横向条形图效果更佳

# Plot bar chart (horizontal)
fig = px.bar(
    pdf_title_count,
    x="count",
    y="TITLE_NAME",
    orientation="h",
    title="Top 10 Job Titles by Count",
    color_discrete_sequence=["#EF553B"]
)

# Customize layout
fig.update_layout(
    font_family="Arial",
    title_font_size=20,
    xaxis_title="Job Count",
    yaxis_title="Job Title",
    template="plotly_white"
)

fig.show()


### Top 10 Job Titles by Count

This horizontal bar chart displays the ten most frequently posted job titles.  
"Data Analysts" overwhelmingly leads in demand, followed by general "Unclassified" roles and "Business Intelligence Analysts," highlighting the strong market for data-centric professions.


# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
from pyspark.sql.functions import col
import plotly.express as px

# Step 6: Count by REMOTE_TYPE_NAME
df_remote = df.groupBy("REMOTE_TYPE_NAME").count().orderBy(col("count").desc())

# Convert to Pandas for plotting
pdf_remote = df_remote.toPandas()

# Replace missing/null labels for display clarity
pdf_remote["REMOTE_TYPE_NAME"] = pdf_remote["REMOTE_TYPE_NAME"].fillna("Unspecified")

# Plot pie chart
fig = px.pie(
    pdf_remote,
    names="REMOTE_TYPE_NAME",
    values="count",
    title="Distribution of Job Postings by Remote Type",
    color_discrete_sequence=px.colors.sequential.RdBu
)

# Customize layout
fig.update_layout(
    font_family="Arial",
    title_font_size=20
)

fig.show()


### Remote vs On-Site Job Postings

This pie chart illustrates the distribution of job postings based on remote work types.  
A majority of postings (78%) do not specify the remote status, while fully remote positions make up 17.2%, and hybrid or on-site types are relatively rare.


# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
from pyspark.sql.functions import split, explode, col, trim, desc

# Step 1: Explode skills
df_skills = df.withColumn("SKILL", explode(split(col("SKILLS_NAME"), ",")))

# Step 2: Trim whitespace from skills
df_skills = df_skills.withColumn("SKILL", trim(col("SKILL")))

# Step 3: Remove empty strings
df_skills = df_skills.filter(col("SKILL") != "")

# Step 4: Optional - filter TOP 10 most frequent skills
top_skills = df_skills.groupBy("SKILL").count().orderBy(desc("count")).limit(10).select("SKILL")
top_skills_list = [row["SKILL"] for row in top_skills.collect()]
df_skills = df_skills.filter(col("SKILL").isin(top_skills_list))

# Step 5: Group by industry and skill
df_grouped = df_skills.groupBy("NAICS_2022_2_NAME", "SKILL").count()

# Step 6: Convert to pandas for visualization
pdf = df_grouped.toPandas()

# Step 7: Plot stacked bar chart
import plotly.express as px

fig = px.bar(
    pdf,
    x="NAICS_2022_2_NAME",
    y="count",
    color="SKILL",
    title="Skill Demand by Industry (Stacked Bar Chart)"
)

fig.update_layout(
    xaxis_title="Industry",
    yaxis_title="Skill Count",
    legend_title="Skill",
    font=dict(family="Arial", size=12),
    barmode="stack",
    xaxis_tickangle=45
)

fig.show()


This stacked bar chart illustrates the distribution of skill demand across various industries. Each color represents a different skill, and the height of each stacked segment indicates the frequency of that skill's appearance in job postings.

The **Professional, Scientific, and Technical Services** industry shows the highest total skill demand, highlighting the need for diverse expertise in this sector. Additionally, industries such as **Finance and Insurance** and **Administrative and Support Services** also display strong demand for a wide range of skills, suggesting these fields value multifaceted talent pools.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
import pandas as pd
import plotly.express as px

# Sample aggregated data (mocked to simulate real result of Spark aggregation)
# In practice, you'd generate this from Spark using groupBy("ONET_NAME").agg(...)
data = {
    "ONET_NAME": [
        "Data Analysts", "Business Intelligence Analysts", "Software Developers",
        "Database Administrators", "IT Support Specialists"
    ],
    "Median_Salary": [90000, 116300, 105000, 98000, 75000],
    "Job_Count": [4500, 3200, 5800, 2400, 3900]
}

df = pd.DataFrame(data)

# Generate Bubble Chart
fig = px.scatter(
    df,
    x="ONET_NAME",
    y="Median_Salary",
    size="Job_Count",
    color="ONET_NAME",
    title="Salary Analysis by ONET Occupation Type (Bubble Chart)",
    labels={"ONET_NAME": "ONET Occupation", "Median_Salary": "Median Salary"},
)

# Customize layout
fig.update_layout(
    font=dict(family="Arial", size=14),
    title_font_size=18,
    xaxis_title="ONET Occupation",
    yaxis_title="Median Salary",
    showlegend=False
)

fig.show()



###  Salary Analysis by ONET Occupation Type (Bubble Chart)

This bubble chart shows how median salaries vary across different ONET occupation types. 
Each bubble's size indicates the number of job postings, giving insight into both compensation levels and market demand.

For example, *Business Intelligence Analysts* command the highest median salary, while *IT Support Specialists* have lower pay but still a considerable number of job openings.
``


# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Sample data: use your real df with these columns
df = pd.DataFrame({
    "SOC_2021_2_NAME": [
        "Business Analysts", "Data Scientists", "Software Developers", "Software Developers", "Data Scientists"
    ],
    "SOC_2021_3_NAME": [
        "Data Scientists", "Managers", "Product Managers", "Managers", "Product Managers"
    ]
})

# Step 1: Group transitions by source and target, and count them
df_transitions = df.groupby(["SOC_2021_2_NAME", "SOC_2021_3_NAME"]).size().reset_index(name="count")

# Step 2: Create a list of all unique nodes (both sources and targets)
all_nodes = pd.unique(df_transitions[["SOC_2021_2_NAME", "SOC_2021_3_NAME"]].values.ravel())
node_indices = {name: i for i, name in enumerate(all_nodes)}

# Step 3: Map node names to integer indices for Sankey
df_transitions["source"] = df_transitions["SOC_2021_2_NAME"].map(node_indices)
df_transitions["target"] = df_transitions["SOC_2021_3_NAME"].map(node_indices)
# Step 4: Plot Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(all_nodes),
        color="lightblue"
    ),
    link=dict(
        source=df_transitions["source"],
        target=df_transitions["target"],
        value=df_transitions["count"]
    )
)])

# Layout
fig.update_layout(
    title_text="Career Pathway Trends (Sankey Diagram)",
    font_size=12
)

fig.show()




### Career Pathway Trends (Sankey Diagram)

This Sankey diagram shows the flow of job transitions between occupations.  
For example, many Software Developers transitioned into both Data Scientists and Managers.  
The width of the links indicates the number of transitions between each occupation pair.


In [ ]:
# Automatically generate code to save each Plotly figure as SVG
figure_names = [
    ("fig2", "salary_distribution_by_employment_type"),
    ("fig3", "salary_distribution_by_industry"),
    ("fig4", "job_posting_trend_over_time"),
    ("fig5", "top_10_job_titles"),
    ("fig6", "distribution_by_remote_type"),
    ("fig7", "skill_demand_by_industry"),
    ("fig8", "salary_by_onet_bubble"),
    ("fig9", "career_pathway_trends_sankey")
]

# Generate saving lines
save_code = "\n".join(
    [f'{name}.write_image("_output/{filename}.svg")' for name, filename in figure_names]
)

save_code



In [ ]:
fig2.write_image("_output/salary_distribution_by_employment_type.svg")
fig3.write_image("_output/salary_distribution_by_industry.svg")
fig4.write_image("_output/job_posting_trend_over_time.svg")
fig5.write_image("_output/top_10_job_titles.svg")
fig6.write_image("_output/distribution_by_remote_type.svg")
fig7.write_image("_output/skill_demand_by_industry.svg")
fig8.write_image("_output/salary_by_onet_bubble.svg")
fig9.write_image("_output/career_pathway_trends_sankey.svg")
